In [1]:
from newspaper import Article
import numpy as np  
import newspaper

## get urls and keywords from file

In [2]:
with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url.replace('\n', '') for url in urls]
urls

['https://vnexpress.net/', 'https://www.24h.com.vn/', 'https://tuoitre.vn/']

In [3]:
with open('keywords.txt', 'r') as f:
    keywords = f.readlines()
keywords = [key.replace('\n', '') for key in keywords]
keywords

['thủ tướng', 'chính phủ', 'việt nam', 'Phạm Minh Chính', 'Nhà nước']

### standardize keywords

In [4]:
keywords = [key.replace(' ', '_') for key in keywords]
keywords = [key.lower() for key in keywords]
list_key_tmp = []
for key in keywords:
    list_key_tmp += key.split('_')

# keywords += list_key_tmp
keywords

['thủ_tướng', 'chính_phủ', 'việt_nam', 'phạm_minh_chính', 'nhà_nước']

## crawl function

In [5]:
from bs4 import BeautifulSoup
import requests
import re  

def is_valid(url):
    # return re.findall(r'\.[a-z]{3}', url)
    return re.findall(r'\.[a-z]{3}', url) and re.search(r'https://', url)

# global dict
dict_url = {}

def get_link_articles_from_url(url):
    respone = requests.get(url)
    soup = BeautifulSoup(respone.text, 'html.parser')
    
    for a in soup.find_all('a', href=True):
        articles_link = a['href'].replace('#box_comment_vne', '')
        if articles_link not in dict_url.keys() and is_valid(articles_link):
            dict_url[articles_link] = 1

    return list(dict_url.keys())

# test 
tmp = get_link_articles_from_url(urls[0])
print(len(tmp))
print(*tmp[:5], sep='\n')


27
https://e.vnexpress.net/
https://video.vnexpress.net
https://vnexpress.net/trinh-mien-nhiem-13-thanh-vien-chinh-phu-4259027.html
https://vnexpress.net/tong-giam-doc-doanh-nghiep-xang-o-sai-gon-bi-bat-4259337.html
https://vnexpress.net/hien-truong-ho-tu-than-rong-hon-100-m2-4259284.html


In [6]:
import newspaperedited

def get_category(url_home):
    dict_cate = {}
    home = newspaperedited.build(url_home)
    category_urls = home.category_urls()
    return list(set(category_urls))

list_cate = [get_category(url) for url in urls]

In [7]:
list_categorys = []
for i in list_cate:
    list_categorys += i  
list_categorys

['https://vnexpress.net/y-kien',
 'https://vnexpress.net/suc-khoe',
 'https://vnexpress.net/doi-song',
 'https://vnexpress.net/so-hoa',
 'https://vnexpress.net/the-gioi',
 'https://vnexpress.net/tin-tuc-24h',
 'https://vnexpress.net/the-thao',
 'https://vnexpress.net/kinh-doanh',
 'https://vnexpress.net/giai-tri',
 'https://e.vnexpress.net',
 'https://vnexpress.net/hai',
 'https://vnexpress.net/thoi-su',
 'https://video.vnexpress.net',
 'https://vnexpress.net/khoa-hoc',
 'https://vnexpress.net/giao-duc',
 'https://vnexpress.net/goc-nhin',
 'https://vnexpress.net/phap-luat',
 'https://vnexpress.net/tam-su',
 'https://vnexpress.net/du-lich',
 'https://vnexpress.net/',
 'https://vnexpress.net',
 'https://vnexpress.net/oto-xe-may',
 'https://www.24h.com.vn',
 'https://www.24h.com.vn/',
 'http://baogia.24h.com.vn',
 'https://tuoitre.vn',
 'https://tuyensinh.tuoitre.vn',
 'https://tuoitre.vn/phap-luat.htm',
 'https://dulich.tuoitre.vn',
 'https://nhadat.tuoitre.vn',
 'https://quangcao.tuoitr

## get keywords from url

In [8]:
def get_text_url(url):
    text = ''
    url = ''
    try:
        article = Article(url)
        article.download()
        article.parse()
    except:
        return text, url
    text = article.text.replace('\n', '.\n')
    url = article.url
    return text, article.url


In [10]:
from pyvi import ViTokenizer
from collections import Counter

def get_keywords_from_text(text):
    tokens = ViTokenizer.tokenize(text)
    tokens = ViTokenizer.spacy_tokenize(tokens)[0]
    tokens = list(filter(lambda x: len(x)>1, tokens))
    counter_tokens = Counter(tokens)
    counter_tokens = dict(counter_tokens)
    counter_tokens = dict(sorted(counter_tokens.items(), key=lambda x:-x[1]))
    return counter_tokens


In [11]:
def get_important_score(dict_keywords, keys):
    score = 0
    for key in dict_keywords.keys():
        for k in keys:
            if k == key.lower():
                score += dict_keywords[key]
    return score


## ranking articles

In [12]:
list_article_url = []

dict_url = {}
for category in list_categorys:
    list_article_url = get_link_articles_from_url(category)

In [13]:
len(list_article_url)

1281

In [15]:
t = get_text_url(list_article_url[100])
print(list_article_url[100])
print(t)

https://vnexpress.net/nhung-buoi-ca-phe-chet-4258591.html
('', '')


In [16]:
list_text_url = []
cnt = 0
for article_url in list_article_url:
    text, url = get_text_url(article_url)
    if text != '':
        list_text_url.append((text, url))
    else:
        continue

    if cnt%10==0:
        print(len(list_text_url), end='-')
    cnt += 1

In [17]:
print(len(list_text_url))

0


In [ ]:
dict_index_and_score = {i:0 for i in range(len(list_text_url))}
# dict_index_and_score {0:0, 1:0, 2:0, .....}
dict_index_and_score

{}

In [ ]:
list_keys = []
cnt = 0
for text, url in list_text_url:
    
    keys = get_keywords_from_text(text)
    # print(keys)
    dict_index_and_score[cnt] = get_important_score(keys, keywords)
    if cnt%10==0:
        print(cnt, end='-')
    cnt += 1


In [ ]:
list_index = dict(sorted(dict_index_and_score.items(), key=lambda x : -x[1]))

In [ ]:
cnt = 0 
for i in list_index:
    cnt+=1 
    print(i, list_index[i])
    print(list_text[i][1])
    if cnt == 10:
        break

In [ ]:
# list_article_url
# print(list_text[557])
# tmp1 = get_keywords_from_text(list_text[557])
# print(tmp1)
# get_important_score(tmp1, ['nhà', 'covid'])